In [1]:
from nl2sql import datasets

from nl2sql.executors.linear_executor.core import CoreLinearExecutor
from nl2sql.llms.vertexai import text_bison_latest
from nl2sql.datasets import fetch_dataset
import json
import sqlite3
import pandas as pd
from langchain_google_vertexai import VertexAI

##TODO
# use interface for SQL execution
# easy configure different executor
# run on different databases. And the golden dataset.

/home/jupyter/.cache/pypoetry/virtualenvs/nl2sql-cA2Qvzrm-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-10 04:34:56.725 | WARNING  | nl2sql.commons.reporting.fingerprint:user_info:40 - 'Credentials' object has no attribute 'service_account_email'
2024-05-10 04:34:57.668 | WARNING  | nl2sql.commons.reporting.fingerprint:user_info:71 - 'email'


In [24]:
from google.cloud import bigquery

BQ_PROJECT_ID = 'proj-kous'
BQ_DATASET_ID = 'Albtertsons'

client = bigquery.Client(project=BQ_PROJECT_ID)
job_config = bigquery.QueryJobConfig(
    maximum_bytes_billed=100000000,
    default_dataset=f'{BQ_PROJECT_ID}.{BQ_DATASET_ID}'
    )

query = "SELECT * FROM `proj-kous.Albertsons.camain_ps` LIMIT 1"

In [25]:
query_job = client.query(query, job_config=job_config)

In [26]:
try:
    response = query_job.result()
    print('success')
except Exception as e:
    print(f'fail {e}')

success


In [39]:
def main1():
    try:
        a = '1' + 1
        print('success')
    except:
        print('fail')

In [40]:
main1()

fail


In [42]:
"He"

'He'

In [38]:
main.__code__

<code object main at 0x174b01df0, file "/var/folders/9r/dk59vr653tdbtv5rf25js0sh011w00/T/ipykernel_48487/126221279.py", line 1>

In [ ]:
llm = VertexAI(temperature=0, model_name="gemini-pro", max_output_tokens=1024)

In [ ]:
def auto_verify(nl_description, ground_truth, llm_amswer):
    """
    This function verifies the accuracy of SQL query based on a natural language description
    and a ground truth query, using text-bison model.

    Parameters:
    - nl_description (str): The natural language description of the SQL query.
    - ground_truth (str): The ground truth SQL query.
    - llm_amswer (str): The student's generated SQL query for validation.

    Returns:
    str: "Yes" if the student's answer matches the ground truth and fits the NL description correctly,
         "No" otherwise.
    """
    
    prompt = f'''You are an expert at validating SQL queries. Given the Natrual language description
      and the SQL query corresponding to that description, please check if the students answer is correct.
      There can be different ways to achieve the same result by forming the query differently.
      If the students SQL query matches the ground truth and fits the NL description correctly, then return yes
      else return no.
      Natural language description: {nl_description}
      Ground truth: {ground_truth}
      students answer: {llm_amswer}
    '''
    return llm(prompt)

In [5]:
bq_project_id = 'vertexai-pgt'
bq_dataset_id = 'nl2sql_fiserv'
ground_truth_path = './fiserv_ground_truth.csv'

In [41]:
def bq_evaluator(bq_project_id, bq_dataset_id, ground_truth_path):
    bigquery_connection_string = f"bigquery://{bq_project_id}/{bq_dataset_id}"
    executor = CoreLinearExecutor.from_connection_string_map(
        {
            bq_dataset_id: bigquery_connection_string
        }
    )
    df = pd.read_csv(ground_truth_path)
    out = []
    for _, (table, question, ground_truth_sql) in df.iterrows():
        result = executor(
            db_name= bq_dataset_id,
            question = question
        )

        generated_query = result.generated_query

        actual_query_class = lambda: None
        actual_query_class.generated_query = ground_truth_sql

        generated_query_result = executor.fetch_result(result)
        actual_query_result = executor.fetch_result(actual_query_class)
        
        llm_rating = auto_verify(question, ground_truth_sql, generated_query)
        result_eval = 0
        if generated_query_result.equals(actual_query_result):
            result_eval = 1

        out.append((question, ground_truth_sql, actual_query_result, generated_query,
                    generated_query_result, llm_rating, result_eval))
        
    df = pd.DataFrame(out, columns=['question', 'ground_truth_sql', 'actual_query_result',
                                    'generated_query', 'generated_query_result', 'query_eval', 'result_eval'
                                    ])
    ts = time.strftime("%y%m%d%H%M")
    df.to_csv(f'eval_output/eval_result_{ts}.csv', index=False)
    print(f'Accuracy: {df.result_eval.sum()/len(df)}')
    return df

In [ ]:
bq_evaluator(bq_project_id, bq_dataset_id, ground_truth_path)

In [2]:
# It will download dataset in /var/tmp/NL2SQL_SPIDER_DATASET/extracted/spider/ folder
spider_dataset_path = "/var/tmp/NL2SQL_SPIDER_DATASET/extracted/spider/"

#For evaluating on test databases:
spider_db_path = spider_dataset_path + "test_database"
spider_eval_json = spider_dataset_path + "test_data/dev.json"

In [3]:
# Spider Evaluator Module
def spider_evaluator(spider_db_path, spider_eval_json, ExecutorType,
                      eval_output_file_name = "eval_results.csv",
                     eval_limit = 100, continuous_failure_limit=1):
    f = open(spider_eval_json)
    eval_data = json.load(f)
    total_no_of_questions = 0
    total_no_of_successfully_executed_questions = 0
    total_no_of_questions_for_which_result_is_correct = 0
    no_of_continuous_failures = 0
    eval_output = []
    for data in eval_data:
        print("\n")
        if total_no_of_questions >= eval_limit or no_of_continuous_failures>=continuous_failure_limit:
            break
        total_no_of_questions +=1
        try:
            db_id = data["db_id"]
            query = data["query"]
            question = data["question"]
            print(question)
            executor = ExecutorType.from_connection_string_map(
                {
                "spider": f"sqlite:///{spider_db_path}/{db_id}/{db_id}.sqlite"
                }
            )

            result = executor(
                db_name= "spider",
                question = question
            )
            generated_query = result.generated_query

            actual_query = query
            print("Actual Query:", actual_query)
            print("Generated Query:", generated_query)
            connection = sqlite3.connect(f"{spider_db_path}/{db_id}/{db_id}.sqlite")
            cursor = connection.cursor()
            generated_query_result = cursor.execute(actual_query).fetchall()
            print("generated_query_result:",generated_query_result)
            actual_query_result = cursor.execute(actual_query).fetchall()
            
            print("actual_query_result:",actual_query_result)
            correct_result = False
            if generated_query_result == actual_query_result:
                total_no_of_questions_for_which_result_is_correct +=1
                correct_result = True
            eval_output.append((db_id,question,actual_query,generated_query,correct_result))
            total_no_of_successfully_executed_questions +=1
            no_of_continuous_failures = 0
        except Exception as e:
            print(e)
            no_of_continuous_failures +=1
        print("\n")
    eval_output_df = pd.DataFrame(eval_output, columns=['db_name', 'question', 'actual_query', 'generated_query',
                                         'correct_generated_result ?'])
    eval_output_df.to_csv(eval_output_file_name, index=False)
    success_ratio_for_execution = total_no_of_successfully_executed_questions/total_no_of_questions
    success_ratio_for_result = total_no_of_questions_for_which_result_is_correct/total_no_of_questions
    print("Total_no_of_questions:",total_no_of_questions)
    print("Total_no_of_successfully_executed_questions:",total_no_of_successfully_executed_questions)
    print("Total_no_of_questions_for_which_result_is_correct:",total_no_of_questions_for_which_result_is_correct)
    print("Success ratio for execution:",success_ratio_for_execution )
    print("Success ratio for result:",success_ratio_for_result )

In [ ]:
spider_evaluator(spider_db_path, spider_eval_json, ExecutorType = CoreLinearExecutor, eval_limit = 100)

2024-05-09 09:52:10.095 | INFO     | nl2sql.datasets.base:fetch_schema:268 - [spider] : Fetching Schema ...
2024-05-09 09:52:10.257 | SUCCESS  | nl2sql.datasets.base:fetch_schema:289 - [spider] : Schema Obtained Successfully
2024-05-09 09:52:10.258 | DEBUG    | nl2sql.datasets.base:from_connection_string:303 - [spider] : Analysing ...
2024-05-09 09:52:10.269 | SUCCESS  | nl2sql.datasets.base:from_connection_string:319 - [spider] : Analysis Complete
2024-05-09 09:52:10.270 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [spider] : Instantiating ...
2024-05-09 09:52:10.271 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [spider] : Calculating Exclusions ...
2024-05-09 09:52:10.272 | INFO     | nl2sql.datasets.base:model_post_init:375 - [spider] : No tables will be excluded
2024-05-09 09:52:10.273 | SUCCESS  | nl2sql.datasets.base:model_post_init:376 - [spider] : Exclusions Calculated
2024-05-09 09:52:10.274 | DEBUG    | nl2sql.datasets.base:model_post_init:377 - [spider] 



How many clubs are there?


2024-05-09 09:52:10.729 | SUCCESS  | nl2sql.datasets.base:model_post_init:488 - [spider] : Custom Descriptions Generated
2024-05-09 09:52:10.730 | SUCCESS  | nl2sql.datasets.base:model_post_init:491 - [spider] : Instantiated
2024-05-09 09:52:13.222 | INFO     | nl2sql.executors.linear_executor.core:__call__:62 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-05-09 09:52:13.223 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-05-09 09:52:13.517 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " club", "generation_info": {"is_blocked": false, "safety_attributes": {"Derogatory": 0.1, "Insult": 0.1, "Sexual": 0.2}}, "type": "Generation"}]], "llm_output": null, "run": [{"run_id": "a3d7cfc1-ee66-4950-841f-5af6e2e6c805"}]}
2024-05-09 09:52:13.519 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [spider

Actual Query: SELECT count(*) FROM club
Generated Query: SELECT COUNT(*) AS "Number_of_Clubs" FROM club;
generated_query_result: [(9,)]
actual_query_result: [(9,)]




Count the number of clubs.


2024-05-09 09:52:20.113 | INFO     | nl2sql.executors.linear_executor.core:__call__:62 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-05-09 09:52:20.114 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-05-09 09:52:20.478 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " club", "generation_info": {"is_blocked": false, "safety_attributes": {"Derogatory": 0.1, "Insult": 0.1, "Sexual": 0.1}}, "type": "Generation"}]], "llm_output": null, "run": [{"run_id": "23a38217-dcc7-49e4-b490-281b6de39415"}]}
2024-05-09 09:52:20.479 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [spider] : Instantiating ...
2024-05-09 09:52:20.480 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [spider] : Calculating Exclusions ...
2024-05-09 09:52:20.481 | INFO     | nl2sql.datasets.base:model_post_init:370 - [spid

...

2024-05-09 09:59:43.275 | INFO     | nl2sql.executors.linear_executor.core:__call__:62 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-05-09 09:59:43.276 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-05-09 09:59:43.638 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " player", "generation_info": {"is_blocked": false, "safety_attributes": {"Derogatory": 0.1, "Finance": 0.6, "Insult": 0.2, "Sexual": 0.1}}, "type": "Generation"}]], "llm_output": null, "run": [{"run_id": "b229f661-9951-44fa-a9dd-82f3df79a6f4"}]}
2024-05-09 09:59:43.639 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [spider] : Instantiating ...
2024-05-09 09:59:43.640 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [spider] : Calculating Exclusions ...
2024-05-09 09:59:43.641 | INFO     | nl2sql.datasets.base:model_pos

Actual Query: SELECT Earnings FROM player WHERE Country  =  "Australia" OR Country  =  "Zimbabwe"
Generated Query: SELECT "Earnings" FROM player WHERE "Country" = 'Australia' OR "Country" = 'Zimbabwe';
generated_query_result: [(1478557.0,), (1359653.0,)]
actual_query_result: [(1478557.0,), (1359653.0,)]




What are the earnings of players from either of the countries of Australia or Zimbabwe?


2024-05-09 09:59:51.313 | INFO     | nl2sql.executors.linear_executor.core:__call__:62 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-05-09 09:59:51.314 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-05-09 09:59:51.663 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " player", "generation_info": {"is_blocked": false, "safety_attributes": {"Derogatory": 0.1, "Finance": 0.6, "Health": 0.1, "Insult": 0.2, "Sexual": 0.1}}, "type": "Generation"}]], "llm_output": null, "run": [{"run_id": "c940082e-1739-47d7-8aa3-808130981a3a"}]}
2024-05-09 09:59:51.665 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [spider] : Instantiating ...
2024-05-09 09:59:51.666 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [spider] : Calculating Exclusions ...
2024-05-09 09:59:51.667 | INFO     | nl2sql.datasets

Actual Query: SELECT Earnings FROM player WHERE Country  =  "Australia" OR Country  =  "Zimbabwe"
Generated Query: SELECT SUM("Earnings") 
FROM player
WHERE "Country" IN ('Australia', 'Zimbabwe');
generated_query_result: [(1478557.0,), (1359653.0,)]
actual_query_result: [(1478557.0,), (1359653.0,)]




List the id, first name and last name of the customers who both have placed more than 2 orders and have bought at least 3 items.


/home/jupyter/.cache/pypoetry/virtualenvs/nl2sql-cA2Qvzrm-py3.10/lib/python3.10/site-packages/langchain/utilities/sql_database.py:357: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows_result = connection.execute(command)  # type: ignore
2024-05-09 09:59:56.753 | SUCCESS  | nl2sql.datasets.base:model_post_init:488 - [spider] : Custom Descriptions Generated
2024-05-09 09:59:56.754 | SUCCESS  | nl2sql.datasets.base:model_post_init:491 - [spider] : Instantiated
2024-05-09 09:59:59.037 | INFO     | nl2sql.executors.linear_executor.core:__call__:62 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-05-09 09:59:59.038 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-05-09 09:59: